# 02 – Optimisation Basics

## Markowitz Mean-Variance Framework

**Objective**: choose portfolio weights **w** ∈ ℝⁿ to:

$$\min_{\mathbf{w}} \; \mathbf{w}^\top \Sigma \mathbf{w}$$

subject to:

$$\mathbf{1}^\top \mathbf{w} = 1, \quad \mathbf{w} \geq 0, \quad \mathbf{w}^\top \boldsymbol{\mu} \geq \mu_t$$

This is a **convex quadratic programme** (QP) solvable in polynomial time.

**Sharpe Ratio** reformulation: the Dinkelbach variable substitution  
$ \mathbf{y} = \mathbf{w} / [(\boldsymbol{\mu}-r_f)^\top \mathbf{w}] $  
turns the fractional program into a QP.

In [ ]:
import sys; sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import cvxpy as cp

from src.data_handler import load_data
from src.optimizer    import min_variance, max_sharpe, min_variance_scipy, max_sharpe_scipy

TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'JPM', 'SPY']
data    = load_data(TICKERS, '2015-01-01', '2024-12-31', train_end='2023-12-31')
mu, cov = data['mu'], data['cov']
mu

In [ ]:
# ── Global Minimum Variance Portfolio (CVXPY) ─────────────────────────────────
gmvp = min_variance(mu, cov)
print('GMVP')
print(f'  Return   : {gmvp["ret"]:.2%}')
print(f'  Volatility: {gmvp["vol"]:.2%}')
print(f'  Sharpe   : {gmvp["sharpe"]:.3f}')
print()
gmvp['weights'].sort_values(ascending=False)

In [ ]:
# ── Maximum Sharpe Ratio Portfolio (CVXPY) ────────────────────────────────────
msr = max_sharpe(mu, cov, risk_free_rate=0.04)
print('Max Sharpe Ratio Portfolio')
print(f'  Return   : {msr["ret"]:.2%}')
print(f'  Volatility: {msr["vol"]:.2%}')
print(f'  Sharpe   : {msr["sharpe"]:.3f}')
print()
msr['weights'].sort_values(ascending=False)

In [ ]:
# ── Constrained: max 30% per asset ───────────────────────────────────────────
gmvp_c = min_variance(mu, cov, max_weight=0.30)
msr_c  = max_sharpe(mu, cov, risk_free_rate=0.04, max_weight=0.30)
comp = pd.DataFrame({
    'GMVP Unconstrained': gmvp['weights'],
    'GMVP (max 30%)':     gmvp_c['weights'],
    'MSR Unconstrained':  msr['weights'],
    'MSR (max 30%)':      msr_c['weights'],
}).fillna(0)
comp.style.format('{:.2%}')

In [ ]:
# ── CVXPY vs SciPy comparison ─────────────────────────────────────────────────
gmvp_sp = min_variance_scipy(mu, cov)
msr_sp  = max_sharpe_scipy(mu, cov, risk_free_rate=0.04)

comparison = pd.DataFrame({
    'GMVP (CVXPY)':  {'Return': gmvp['ret'], 'Vol': gmvp['vol'], 'Sharpe': gmvp['sharpe']},
    'GMVP (SciPy)':  {'Return': gmvp_sp['ret'], 'Vol': gmvp_sp['vol'], 'Sharpe': gmvp_sp['sharpe']},
    'MSR  (CVXPY)':  {'Return': msr['ret'], 'Vol': msr['vol'], 'Sharpe': msr['sharpe']},
    'MSR  (SciPy)':  {'Return': msr_sp['ret'], 'Vol': msr_sp['vol'], 'Sharpe': msr_sp['sharpe']},
}).T
comparison.style.format({'Return': '{:.2%}', 'Vol': '{:.2%}', 'Sharpe': '{:.4f}'})

In [ ]:
# ── Raw CVXPY example (pedagogical) ──────────────────────────────────────────
# Shows the bare-metal QP formulation
mu_arr  = mu.values
cov_arr = cov.values
n       = len(mu_arr)

w = cp.Variable(n)

# Objective: minimise portfolio variance  wᵀΣw
objective = cp.Minimize(cp.quad_form(w, cov_arr))

# Constraints: fully invested, long-only
constraints = [
    cp.sum(w) == 1,   # budget constraint
    w >= 0,           # no short selling
]

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.CLARABEL)

print('Solver status:', prob.status)
print('Optimal weights:')
for ticker, weight in zip(TICKERS, w.value):
    print(f'  {ticker}: {weight:.4f}')